In [1]:
import pandas as pd
import numpy as np
     
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize 
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

!pip install emoji
import emoji

nltk.download('punkt')
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")
!pip install -q keras
import keras
from os import path
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import Sequential
from keras import layers
from google.colab import drive
drive.mount('/content/drive')
import csv



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 10.1 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=0237b1fef44b153477f66777c1ea77318dcfee4cefbf32ee98258b7c8ff85a2e
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


cu80
Mounted at /content/drive


## Data Reading

In [ ]:
# 导入训练集数据 土耳其语
olid_training=pd.read_csv("/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-training-v1.tsv",sep="\t")
X_train_FULL=olid_training[["id","tweet","subtask_a"]] 
Y_train_FULL=olid_training["subtask_a"]
# 导入测试数据及结果
X_test_FULL=pd.read_csv('/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-testset-v1.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TEST=pd.read_csv('/content/drive/MyDrive/TURKISH-DATA/offenseval-tr-labela-v1.tsv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

print("---导入数据成功---")



---导入数据成功---


In [10]:
# 导入训练集数据 英语
olid_training=pd.read_csv("/content/drive/MyDrive/OLIDv1.0/olid-training-v1.0.tsv",sep="\t")
X_train_FULL=olid_training[["id","tweet","subtask_a"]] 
Y_train_FULL=olid_training["subtask_a"]
# 导入测试数据及结果
X_test_FULL=pd.read_csv('/content/drive/MyDrive/OLIDv1.0/testset-levela.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TEST=pd.read_csv('/content/drive/MyDrive/OLIDv1.0/labels-levela.csv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

print("---导入数据成功---")


---导入数据成功---


Pre-processing

In [3]:
def preprocess_tweet(tweet):
    # Split tweet into tokens
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(tweet)

    # Remove URLs and mentions
    tokens = [token for token in tokens if not token.startswith('http') and not token.startswith('@')]

    # Decode emojis
    tokens = [emoji.demojize(token) for token in tokens]

    # Remove punctuation and non-alphanumeric characters
    tokens = [token for token in tokens if token.isalnum()]

    # Convert all tokens to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    tokens = [token for token in tokens if ((token != "url"))]
    return tokens

In [11]:
# 数据清洗

filtered_tweets=[]
for tweet in X_train_FULL["tweet"]:
    filtered_tweets.append(preprocess_tweet(tweet))
X_train_FULL["tweet_initial"] = filtered_tweets


filtered_tweets=[]
for tweet in X_test_FULL["tweet"]:
    filtered_tweets.append(preprocess_tweet(tweet))
X_test_FULL["tweet_initial"] = filtered_tweets




#FOR TEST

z=[]
for tweet in X_train_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_train_FULL["tweet_initial_nontoken"]=z



#FOR TEST

z=[]
for tweet in X_test_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_test_FULL["tweet_initial_nontoken"]=z



In [12]:
X_test_FULL

,id,tweet,tweet_initial,tweet_initial_nontoken
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,"[democrats, support, antifa, muslim, brotherho...",democrats support antifa muslim brotherhood ms...
1,27014,"#ConstitutionDay is revered by Conservatives, ...","[revered, conservatives, hated, progressives, ...",revered conservatives hated progressives socia...
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,"[first, reduces, ca]",first reduces ca
3,13876,#Watching #Boomer getting the news that she is...,"[getting, news, still, parole, always, makes, ...",getting news still parole always makes smile f...
4,60133,#NoPasaran: Unity demo to oppose the far-right...,"[unity, demo, oppose, enough, enough]",unity demo oppose enough enough
...,...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,"[lie, rifles, dem, distorted, law, push, kavan...",lie rifles dem distorted law push kavanaugh co...
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,"[present, event, oiw, 2018, finpact, global, i...",present event oiw 2018 finpact global impact f...
857,67018,3 people just unfollowed me for talking about ...,"[3, people, unfollowed, talking, merlin, sorry...",3 people unfollowed talking merlin sorry im st...
858,50665,#WednesdayWisdom Antifa calls the right fascis...,"[antifa, calls, right, fascist, reality, left,...",antifa calls right fascist reality left follow...


In [7]:
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers import Embedding
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
def recall_m(true_Y, pred_Y):
        TP = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        possible_pos = K.sum(K.round(K.clip(true_Y, 0, 1)))
        rec = TP / (possible_pos + K.epsilon())
        return rec

def precision_m(true_Y, pred_Y):
        true_positives = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(pred_Y, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision


    

def f1_m(true_Y, pred_Y):
    pres = precision_m(true_Y, pred_Y)
    rec = recall_m(true_Y, pred_Y)
    return 2*((pres*rec)/(pres+rec+K.epsilon()))

from keras.models import Sequential
from keras import layers


## Tweeter Word2vec / Custom Embedding Layer

In [ ]:
fname= "/content/drive/My Drive/Twitter/Word2Vec/w2v_model_word.vec"


import gensim
from gensim.models import Word2Vec
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname)  # you can load this saved keyedvectors model later



# Tweeter FastText

In [ ]:
import gensim
from gensim.models import FastText

word_vectors = gensim.models.FastText.load_fasttext_format('/content/drive/My Drive/Twitter/FastText/fastText_25022020.bin',encoding='utf-8') # use that if you want to use fasttedxt 



# Public FastText

In [ ]:
word_vectors = gensim.models.FastText.load_fasttext_format('/content/drive/My Drive/OFFENSEVAL20-DATA/haber-P1_S0_L0.bin',encoding='utf-8') # use that if you want to use fasttedxt 


# Public word2vec

In [8]:

fname= "/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz"

import gensim
from gensim.models import Word2Vec

word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname, binary = True)


## Tokenizing / creating vocabulary and wordindex using keras functinalities

In [13]:
"""
We will use word indexes as look-up table during embedding layer.
"""
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()  #the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
# tokenizer = Tokenizer(num_words=98790)  #the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
tokenizer.fit_on_texts(X_train_FULL["tweet_initial_nontoken"])
X_train_initial = tokenizer.texts_to_sequences(X_train_FULL["tweet_initial_nontoken"])
X_test_initial = tokenizer.texts_to_sequences(X_test_FULL["tweet_initial_nontoken"])
vocab_size_initial = len(tokenizer.word_index) + 1 
wordIndex_initial=tokenizer.word_index # it is  index
from keras_preprocessing.sequence import pad_sequences
max_len = 30

"""
Padding

"""
X_train_initial = pad_sequences(X_train_initial, padding='post', maxlen=max_len)
X_test_initial = pad_sequences(X_test_initial, padding='post', maxlen=max_len)




In [14]:
def createEmbeddingLayer(wordIndex,not_static):
  a=[]
  embedding_dim=300
  vocabulary_size=len(wordIndex)+1
  embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
  missed=0
  for word, i in wordIndex.items():
    
          
      try:
          embedding_vector = word_vectors[word] # or fast text
          embedding_matrix[i] = embedding_vector
            

      except KeyError: # If word is not found in the word2vec vocabulary , assign random weights
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)
        missed+=1
        a.append(word)

  print('missed_words :' , missed)

  custom_embedding_layer = Embedding(vocabulary_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                trainable=not_static )# Controls the updating weights )
  return custom_embedding_layer

  